# Download Histone Mod. and TFs data from ENCODE

In order to run Avocado we need to feed the method with Histone and TF data.

- [ ] Download data from http://encodeproject.org
- [ ] Upload data to Synapse


The following query was used to retrieve signal bigWigs from the ENCODE project portal:

https://www.encodeproject.org/metadata/searchTerm=chip-seq&type=Experiment&status=released&biosample_ontology.term_name=K562&assay_title=ChIP-seq&biosample_ontology.classification=cell+line&files.file_type=bigWig&replicates.library.biosample.treatments.treatment_term_name%21=interferon+alpha&replicates.library.biosample.treatments.treatment_term_name%21=Interferon+gamma&target.investigated_as%21=control&target.investigated_as=histone&target.investigated_as=broad+histone+mark&target.investigated_as=narrow+histone+mark/metadata.tsv


Create necessary folders

In [75]:
%%bash
mkdir -p /data/reddylab/projects/encode4_element_selection/data/bigwigs
mkdir -p /data/reddylab/projects/encode4_element_selection/logs
mkdir -p /data/reddylab/projects/encode4_element_selection/metadata

Download metadata selecting for combined (replicates 1 and 2 combined) histone mods. p-val signal files 

In [67]:
%%bash
mkdir -p /data/reddylab/projects/encode4_element_selection/data
curl -SLs "https://www.encodeproject.org/metadata/searchTerm=chip-seq&type=Experiment&status=released&biosample_ontology.term_name=K562&assay_title=ChIP-seq&biosample_ontology.classification=cell+line&files.file_type=bigWig&replicates.library.biosample.treatments.treatment_term_name%21=interferon+alpha&replicates.library.biosample.treatments.treatment_term_name%21=Interferon+gamma&target.investigated_as%21=control&target.investigated_as=histone&target.investigated_as=broad+histone+mark&target.investigated_as=narrow+histone+mark/metadata.tsv" \
    | /bin/grep "1, 2" \
    | /bin/grep "signal p-value" \
    | /bin/grep "hg19" \
    | /bin/grep "released" \
    | awk -F"\t" -vOFS="\t" '{print $7"."$19"."$1, $43, $26}' \
    | sort -k1,1 -k3,3 \
> /data/reddylab/projects/encode4_element_selection/data/metadata.selected_histones.txt

Because there are some factors that have more than a single option, select the **most recent** choice, manually commenting the undesired ones

In [68]:
%%writefile /data/reddylab/projects/encode4_element_selection/data/metadata.selected_histones.txt
K562.H2AFZ-human.ENCFF169KVV	https://www.encodeproject.org/files/ENCFF169KVV/@@download/ENCFF169KVV.bigWig	2011-02-10
K562.H3K27ac-human.ENCFF840LLW	https://www.encodeproject.org/files/ENCFF840LLW/@@download/ENCFF840LLW.bigWig	2011-02-10
K562.H3K27me3-human.ENCFF552AKP	https://www.encodeproject.org/files/ENCFF552AKP/@@download/ENCFF552AKP.bigWig	2011-05-19
# K562.H3K27me3-human.ENCFF658JMW	https://www.encodeproject.org/files/ENCFF658JMW/@@download/ENCFF658JMW.bigWig	2011-02-10
# K562.H3K36me3-human.ENCFF464YSK	https://www.encodeproject.org/files/ENCFF464YSK/@@download/ENCFF464YSK.bigWig	2011-02-10
K562.H3K36me3-human.ENCFF958BZN	https://www.encodeproject.org/files/ENCFF958BZN/@@download/ENCFF958BZN.bigWig	2011-02-11
K562.H3K4me1-human.ENCFF012JPC	https://www.encodeproject.org/files/ENCFF012JPC/@@download/ENCFF012JPC.bigWig	2011-10-07
# K562.H3K4me1-human.ENCFF900IMQ	https://www.encodeproject.org/files/ENCFF900IMQ/@@download/ENCFF900IMQ.bigWig	2011-02-10
K562.H3K4me2-human.ENCFF106RCF	https://www.encodeproject.org/files/ENCFF106RCF/@@download/ENCFF106RCF.bigWig	2011-02-10
# K562.H3K4me3-human.ENCFF029DFM	https://www.encodeproject.org/files/ENCFF029DFM/@@download/ENCFF029DFM.bigWig	2011-02-10
# K562.H3K4me3-human.ENCFF130CUR	https://www.encodeproject.org/files/ENCFF130CUR/@@download/ENCFF130CUR.bigWig	2011-02-11
K562.H3K4me3-human.ENCFF285EUA	https://www.encodeproject.org/files/ENCFF285EUA/@@download/ENCFF285EUA.bigWig	2016-11-08
# K562.H3K4me3-human.ENCFF452UGM	https://www.encodeproject.org/files/ENCFF452UGM/@@download/ENCFF452UGM.bigWig	2011-05-19
K562.H3K79me2-human.ENCFF677DJM	https://www.encodeproject.org/files/ENCFF677DJM/@@download/ENCFF677DJM.bigWig	2011-02-10
K562.H3K9ac-human.ENCFF235MFR   https://www.encodeproject.org/files/ENCFF235MFR/@@download/ENCFF235MFR.bigWig   2011-05-19
# K562.H3K9ac-human.ENCFF957FQF   https://www.encodeproject.org/files/ENCFF957FQF/@@download/ENCFF957FQF.bigWig   2011-02-10
K562.H3K9me3-human.ENCFF330EOT  https://www.encodeproject.org/files/ENCFF330EOT/@@download/ENCFF330EOT.bigWig   2011-02-10
K562.H4K20me1-human.ENCFF359JVI https://www.encodeproject.org/files/ENCFF359JVI/@@download/ENCFF359JVI.bigWig   2011-02-10


Overwriting /data/reddylab/projects/encode4_element_selection/data/metadata.selected_histones.txt


Download bigWigs

In [72]:
%%bash
OFS="\t"
OUTDIR="/data/reddylab/projects/encode4_element_selection/data/bigwigs"
while read -r SAMPLE_NAME SAMPLE_URL;
do
    if [ ${SAMPLE_NAME} != "#" ]; then
        sbatch -pnew,all -o /data/reddylab/projects/encode4_element_selection/logs/${SAMPLE_NAME}.out \
            --wrap="curl -SLo ${OUTDIR}/${SAMPLE_NAME}.bw ${SAMPLE_URL}"
    fi
done </data/reddylab/projects/encode4_element_selection/data/metadata.selected_histones.txt

Submitted batch job 7929637
Submitted batch job 7929638
Submitted batch job 7929639
Submitted batch job 7929640
Submitted batch job 7929641
Submitted batch job 7929642
Submitted batch job 7929643
Submitted batch job 7929644
Submitted batch job 7929645
Submitted batch job 7929646


In [74]:
!ls -la /data/reddylab/projects/encode4_element_selection/data/bigwigs

total 4329232
drwxr-sr-x+ 2 aeb84 reddylab      8192 Mar 18 15:48 .
drwxr-sr-x+ 3 aeb84 reddylab       512 Mar 18 14:59 ..
-rw-rw-r--  1 aeb84 reddylab 359033131 Mar 18 15:34 K562.H2AFZ-human.ENCFF169KVV.bw
-rw-rw-r--  1 aeb84 reddylab 331243852 Mar 18 15:34 K562.H3K27ac-human.ENCFF840LLW.bw
-rw-rw-r--  1 aeb84 reddylab 601772450 Mar 18 15:35 K562.H3K27me3-human.ENCFF552AKP.bw
-rw-rw-r--  1 aeb84 reddylab 322417925 Mar 18 15:34 K562.H3K36me3-human.ENCFF958BZN.bw
-rw-rw-r--  1 aeb84 reddylab 569337920 Mar 18 15:34 K562.H3K4me1-human.ENCFF012JPC.bw
-rw-rw-r--  1 aeb84 reddylab 324479909 Mar 18 15:35 K562.H3K4me2-human.ENCFF106RCF.bw
-rw-rw-r--  1 aeb84 reddylab 652087879 Mar 18 15:34 K562.H3K4me3-human.ENCFF285EUA.bw
-rw-rw-r--  1 aeb84 reddylab 325295350 Mar 18 15:34 K562.H3K79me2-human.ENCFF677DJM.bw
-rw-rw-r--  1 aeb84 reddylab 580566140 Mar 18 15:34 K562.H3K9ac-human.ENCFF235MFR.bw
-rw-rw-r--  1 aeb84 reddylab 366121438 Mar 18 15:34 K562.H3K9me3-human.ENCFF330EOT.bw


liftOver interest regions from hg38 to hg19 (the regions obtained from the Google spreadsheet are in `hg38` https://docs.google.com/spreadsheets/d/1C4it8W9CdGpZjBHwkYyZjPVs5jFeD4Qbb6mhNRauaFs/edit#gid=398768999)

In [2]:
%%writefile  /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.hg38.bed
chrX	47786554	49786554	GATA1
chr8	126736069	128736069	MYC
chr11	4505604	6505604	HBE1
chr11	32869816	34869816	LMO2
chr20	56391407	58391407	RBM38
chr16	1	1172847	HBA2
chr2	59553498	61553498	BCL11A


Overwriting /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.hg38.bed


In [5]:
%%bash
/data/reddylab/software/bin/liftOver \
    /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.hg38.bed \
    /data/reddylab/Reference_Data/Genomes/hg38/hg38ToHg19.over.chain \
    /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.hg19.bed \
    /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.hg19.unmapped.bed

Reading liftover chains
Mapping coordinates


In [2]:
%%writefile /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.all.hg19.bed
chrX	47644982	49644982	GATA1
chr8	127748314	129748315	MYC
chr11	4526834	6526834	HBE1
chr11	32891362	34891363	LMO2
chr20	54966463	56966463	RBM38
chr16	60000	1222847	HBA2
chr2	59780633	61780633	BCL11A


Writing /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.all.hg19.bed


Use `bwtool window` to do the binning of the signal files in *250bp* windows 

In [5]:
%%bash
mkdir -p /data/reddylab/projects/encode4_element_selection/data/bigwigs
BIGWIGS=($(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/*bw))
module load htslib
sbatch -pnew,all \
    --cpus-per-task 1 \
    --mem 8G \
    --array=0-9 \
    --output=/data/reddylab/projects/encode4_element_selection/logs/bw_to_bins.%a.out \
    <<'EOF'
#!/bin/bash
BIN_SIZE=250
BIGWIGS=($(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/*bw))
BIGWIG=${BIGWIGS[${SLURM_ARRAY_TASK_ID}]}

/data/reddylab/software/bwtool/build/bin/bwtool window \
    ${BIN_SIZE} \
    ${BIGWIG} \
    -step=${BIN_SIZE} \
    -fill=0 \
> ${BIGWIG/.bw/.binsize_${BIN_SIZE}.bdg} \
&& echo "${BIGWIG} Done." \
|| echo "${BIGWIG} Failed!"

EOF


Submitted batch job 7953276


Take the mean signal in each window (bwtool window does not aggregate automatically the values in each window, instead it list all values as a comma-separated list)

In [7]:
%%bash
mkdir -p /data/reddylab/projects/encode4_element_selection/data/bigwigs
BIGWIGS=($(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/*bw))
module load htslib
sbatch -pnew,all \
    --cpus-per-task 1 \
    --mem 8G \
    --array=0-9 \
    --output=/data/reddylab/projects/encode4_element_selection/logs/take_mean_bdgs.%a.out \
    <<'EOF'
#!/bin/bash
BIN_SIZE=250
BIGWIGS=($(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/*bw))
BIGWIG=${BIGWIGS[${SLURM_ARRAY_TASK_ID}]}


awk -vOFS="\t" '{tot=0; split($4, vv, ","); for (ii=1; ii<=length(vv); ii+=1){tot+=vv[ii]} print $1,$2,$3,tot/length(vv)}' \
    ${BIGWIG/.bw/.binsize_${BIN_SIZE}.bdg} \
> ${BIGWIG/.bw/.binsize_${BIN_SIZE}.mean.bdg} \
&& echo "${BIGWIG} Done." \
|| echo "${BIGWIG} Failed!"
EOF


Submitted batch job 7960071


Intersect the bedgraphs previously created with the regions of interest defined in the Google spreadsheet

In [3]:
%%bash
mkdir -p /data/reddylab/projects/encode4_element_selection/data/bigwigs
BIGWIGS=($(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/*bw))
module load htslib
sbatch -pnew,all \
    --cpus-per-task 1 \
    --mem 8G \
    --array=0-9 \
    --output=/data/reddylab/projects/encode4_element_selection/logs/intersect_bdgs_with_loci.%a.out \
    <<'EOF'
#!/bin/bash
BIN_SIZE=250
BIGWIGS=($(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/*bw))
BIGWIG=${BIGWIGS[${SLURM_ARRAY_TASK_ID}]}


bedtools intersect \
    -wa -wb \
    -a ${BIGWIG/.bw/.binsize_${BIN_SIZE}.mean.bdg}  \
    -b /data/reddylab/projects/encode4_element_selection/metadata/regions_of_interest.K562.all.hg19.bed \
> ${BIGWIG/.bw/.binsize_${BIN_SIZE}.mean.in_loci_of_interest.txt} \
&& echo "${BIGWIG/.bw/.binsize_${BIN_SIZE}.mean.in_loci_of_interest.txt} Done." \
|| echo "${BIGWIG/.bw/.binsize_${BIN_SIZE}.mean.in_loci_of_interest.txt} Failed!" \
EOF


Submitted batch job 8305607


Merge individual factors in one single table with regions in the x-axis and p-value signals in the y-axis 

In [4]:
%%bash
for ii in $(/bin/ls -1 /data/reddylab/projects/encode4_element_selection/data/bigwigs/K562.*.binsize_250.mean.in_loci_of_interest.txt  \
    | xargs -I{} basename {} \
    | cut -d. -f2); 
    do 
        printf "\t${ii}"; 
    done > /data/reddylab/projects/encode4_element_selection/data/bigwigs/K562.binsize_250.mean.in_loci_of_interest.txt && \
printf "\n" >> /data/reddylab/projects/encode4_element_selection/data/bigwigs/K562.binsize_250.mean.in_loci_of_interest.txt&& \
paste /data/reddylab/projects/encode4_element_selection/data/bigwigs/K562.*.binsize_250.mean.in_loci_of_interest.txt \
    | awk -vOFS="\t" '{printf $1"_"$2"_"$3"_"$8; for (ii=4; ii<=NF; ii+=8){printf "\t"$ii;} printf "\n"}' \
>> /data/reddylab/projects/encode4_element_selection/data/bigwigs/K562.binsize_250.mean.in_loci_of_interest.txt
